In [1]:
import pandas as pd
import numpy as np

import itertools

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

## Load Data

In [2]:
train_macro = pd.read_csv("~/DATA401-Project3/train_macro.csv")

In [3]:
train_macro.head()

,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,product_type,...,provision_retail_space_modern_sqm,turnover_catering_per_cap,theaters_viewers_per_1000_cap,seats_theather_rfmin_per_100000_cap,museum_visitis_per_100_cap,bandwidth_sports,population_reg_sports_share,students_reg_sports_share,apartment_build,apartment_fund_sqm
0,-0.294873,-0.159255,-0.691912,0.0,0.0,0.0,3.147685e-16,3.794261e-17,'2',Investment,...,-6.282208,-3.26676,-1.51348,1.063886,-0.802971,-1.75854,-2.099857,-1.531171,-4.523617,0.258671
1,-0.531523,-0.331346,-0.880403,0.0,0.0,0.0,3.147685e-16,3.794261e-17,'2',Investment,...,-6.282208,-3.26676,-1.51348,1.063886,-0.802971,-1.75854,-2.099857,-1.531171,-4.523617,0.258671
2,-0.294873,-0.116232,-1.068893,0.0,0.0,0.0,3.147685e-16,3.794261e-17,'2',Investment,...,-6.282208,-3.26676,-1.51348,1.063886,-0.802971,-1.75854,-2.099857,-1.531171,-4.523617,0.258671
3,0.914671,0.335508,0.250541,0.0,0.0,0.0,3.147685e-16,3.794261e-17,'2',Investment,...,-6.282208,-3.26676,-1.51348,1.063886,-0.802971,-1.75854,-2.099857,-1.531171,-4.523617,0.258671
4,0.599138,0.916316,-0.691912,0.0,0.0,0.0,3.147685e-16,3.794261e-17,'2',Investment,...,-6.282208,-3.26676,-1.51348,1.063886,-0.802971,-1.75854,-2.099857,-1.531171,-4.523617,0.258671


## Prepare Feature Sets

Load in all the feature sets from the previous notebook and find out which set performs the best in linear regression.

In [4]:
# train features from notebook 2 using SelectKBest methods
train_feats_2_kbest = ['num_room', 'full_sq', 'sport_count_5000', 'sport_count_3000',
                       'trc_count_5000', 'zd_vokzaly_avto_km', 'sadovoe_km', 'kremlin_km',
                       'bulvar_ring_km', 'sport_count_2000', 'ttk_km', 'office_sqm_5000',
                       'trc_sqm_5000', 'sport_count_1500', 'nuclear_reactor_km',
                       'sport_objects_raion', 'trc_count_3000', 'cafe_count_5000_price_1000',
                       'stadium_km', 'cafe_count_5000_price_1500', 'cafe_count_5000',
                       'cafe_count_5000_na_price', 'ecology_no data',
                       'cafe_count_5000_price_500', 'office_sqm_3000',
                       'cafe_count_5000_price_2500', 'trc_sqm_3000', 'basketball_km',
                       'office_km', 'detention_facility_km', 'office_count_5000',
                       'university_km', 'office_sqm_2000', 'theater_km',
                       'cafe_count_5000_price_high', 'church_count_5000', 'swim_pool_km',
                       'catering_km', 'thermal_power_plant_km', 'cafe_count_5000_price_4000']

train_feats_2_featimp = ["full_sq", "num_room", "life_sq", "kitch_sq", "floor", "state_'4'",
                         "state_'2'", "max_floor", "cafe_count_3000_price_2500",
                         "cafe_count_3000", "product_type_Investment", "trc_count_3000",
                         "cafe_count_2000", "cafe_count_5000_price_1500",
                         "product_type_OwnerOccupier", "cafe_count_5000_price_2500",
                         "cafe_count_5000_price_1000", "cafe_count_3000_price_high",
                         "cafe_count_3000_price_1000", "cafe_count_5000", "build_year",
                         "cafe_count_5000_price_high", "cafe_count_2000_price_2500",
                         "cafe_count_3000_price_1500", "cafe_count_5000_price_4000", "material",
                         "sport_count_3000", "office_count_5000", "office_sqm_5000",
                         "cafe_count_1500_price_2500", "industrial_km",
                         "cafe_count_3000_price_500", "trc_count_5000", "church_count_500",
                         "cafe_count_1000_price_2500", "kindergarten_km",
                         "ID_railroad_station_walk", "cafe_count_2000_price_1000",
                         "cafe_count_2000_price_1500", "sport_count_5000"]

train_feats_2_corr = ['num_room', 'full_sq', 'sport_count_5000', 'sport_count_3000',
                      'trc_count_5000', 'sport_count_2000', 'office_sqm_5000', 'trc_sqm_5000',
                      'sport_count_1500', 'sport_objects_raion', 'trc_count_3000',
                      'cafe_count_5000_price_1000', 'cafe_count_5000_price_1500',
                      'cafe_count_5000', 'cafe_count_5000_na_price',
                      'cafe_count_5000_price_500', 'office_sqm_3000',
                      'cafe_count_5000_price_2500', 'trc_sqm_3000', 'office_count_5000',
                      'office_sqm_2000', 'cafe_count_5000_price_high', 'church_count_5000',
                      'cafe_count_5000_price_4000', 'leisure_count_5000',
                      'big_church_count_5000', 'sport_count_1000', 'office_sqm_1500',
                      'market_count_5000', 'school_education_centers_raion',
                      'healthcare_centers_raion', 'office_count_500', 'trc_count_2000',
                      'ekder_male', 'cafe_count_500_price_1000', 'office_count_1000',
                      'mosque_count_5000', 'ekder_all', 'cafe_count_1000_price_high',
                      'ekder_female']

train_feats_4_kbest = ['num_room', 'full_sq', 'sport_count_5000', 'sport_count_3000', # noticed that this list and train_feats_2_corr are same
                       'trc_count_5000', 'sport_count_2000', 'office_sqm_5000',
                       'trc_sqm_5000', 'sport_count_1500', 'sport_objects_raion',
                       'trc_count_3000', 'cafe_count_5000_price_1000',
                       'cafe_count_5000_price_1500', 'cafe_count_5000',
                       'cafe_count_5000_na_price', 'cafe_count_5000_price_500',
                       'office_sqm_3000', 'cafe_count_5000_price_2500', 'trc_sqm_3000',
                       'office_count_5000', 'office_sqm_2000',
                       'cafe_count_5000_price_high', 'church_count_5000',
                       'cafe_count_5000_price_4000', 'leisure_count_5000',
                       'big_church_count_5000', 'sport_count_1000', 'office_sqm_1500',
                       'market_count_5000', 'school_education_centers_raion',
                       'healthcare_centers_raion', 'office_count_500', 'trc_count_2000',
                       'ekder_male', 'cafe_count_500_price_1000', 'office_count_1000',
                       'mosque_count_5000', 'ekder_all', 'cafe_count_1000_price_high',
                       'ekder_female']

train_feats_4_featimp = ['num_room', 'full_sq', 'sport_count_5000', 'sport_count_3000',
                         'trc_count_5000', 'sport_count_2000', 'office_sqm_5000',
                         'trc_sqm_5000', 'sport_count_1500', 'sport_objects_raion',
                         'trc_count_3000', 'cafe_count_5000_price_1000',
                         'cafe_count_5000_price_1500', 'cafe_count_5000',
                         'cafe_count_5000_na_price', 'cafe_count_5000_price_500',
                         'office_sqm_3000', 'cafe_count_5000_price_2500', 'trc_sqm_3000',
                         'office_count_5000', 'office_sqm_2000',
                         'cafe_count_5000_price_high', 'church_count_5000',
                         'cafe_count_5000_price_4000', 'leisure_count_5000',
                         'big_church_count_5000', 'sport_count_1000', 'office_sqm_1500',
                         'market_count_5000', 'school_education_centers_raion',
                         'healthcare_centers_raion', 'office_count_500', 'trc_count_2000',
                         'ekder_male', 'cafe_count_500_price_1000', 'office_count_1000',
                         'mosque_count_5000', 'ekder_all', 'cafe_count_1000_price_high',
                         'ekder_female']

train_feats_4_corr = ['num_room', 'full_sq', 'sport_count_5000', 'sport_count_3000', # same as train_feats_4_featimp
                      'trc_count_5000', 'sport_count_2000', 'office_sqm_5000',
                      'trc_sqm_5000', 'sport_count_1500', 'sport_objects_raion',
                      'trc_count_3000', 'cafe_count_5000_price_1000',
                      'cafe_count_5000_price_1500', 'cafe_count_5000',
                      'cafe_count_5000_na_price', 'cafe_count_5000_price_500',
                      'office_sqm_3000', 'cafe_count_5000_price_2500', 'trc_sqm_3000',
                      'office_count_5000', 'office_sqm_2000',
                      'cafe_count_5000_price_high', 'church_count_5000',
                      'cafe_count_5000_price_4000', 'leisure_count_5000',
                      'big_church_count_5000', 'sport_count_1000', 'office_sqm_1500',
                      'market_count_5000', 'school_education_centers_raion',
                      'healthcare_centers_raion', 'office_count_500', 'trc_count_2000',
                      'ekder_male', 'cafe_count_500_price_1000', 'office_count_1000',
                      'mosque_count_5000', 'ekder_all', 'cafe_count_1000_price_high',
                      'ekder_female']

all_train_feats = [train_feats_2_kbest, train_feats_2_featimp, train_feats_2_corr, 
                   train_feats_4_kbest, train_feats_4_featimp, train_feats_4_corr]

In [5]:
macro_feats_3_kbest = ['bandwidth_sports', 'fixed_basket', 'provision_doctors', 'cpi',
                       'gdp_annual_growth', 'salary', 'gdp_deflator',
                       'load_of_teachers_school_per_teacher', 'turnover_catering_per_cap',
                       'deposits_value']

macro_feats_3_featimp = ['micex_cbi_tr', 'eurrub', 'micex', 'brent', 'usdrub', 'rts',
                         'micex_rgbi_tr', 'gdp_annual_growth', 'average_life_exp', 'cpi']

macro_feats_3_corr = ['bandwidth_sports', 'fixed_basket', 'cpi', 'salary', 'gdp_deflator',
                      'load_of_teachers_school_per_teacher', 'turnover_catering_per_cap',
                      'deposits_value', 'gdp_annual', 'labor_force']

macro_feats_4_kbest = ['bandwidth_sports', 'fixed_basket', 'cpi', 'salary',
                       'gdp_deflator', 'load_of_teachers_school_per_teacher',
                       'turnover_catering_per_cap', 'deposits_value', 'gdp_annual',
                       'labor_force']

macro_feats_4_featimp = ['bandwidth_sports', 'fixed_basket', 'cpi', 'salary',
                         'gdp_deflator', 'load_of_teachers_school_per_teacher',
                         'turnover_catering_per_cap', 'deposits_value', 'gdp_annual',
                         'labor_force']

macro_feats_4_corr = ['bandwidth_sports', 'fixed_basket', 'cpi', 'salary',
                      'gdp_deflator', 'load_of_teachers_school_per_teacher',
                      'turnover_catering_per_cap', 'deposits_value', 'gdp_annual',
                      'labor_force']

all_macro_feats = [macro_feats_3_kbest, macro_feats_3_featimp, macro_feats_3_corr, 
                   macro_feats_4_kbest, macro_feats_4_featimp, macro_feats_4_corr]

In [6]:
b_set = set(map(tuple,all_train_feats))  # need to convert the inner lists to tuples so they are hashable
final_train_feats_list = list(map(list,b_set)) # now convert tuples back into lists 

b_set = set(map(tuple,all_macro_feats))  # need to convert the inner lists to tuples so they are hashable
final_macro_feats_list = list(map(list,b_set)) # now convert tuples back into lists 

In [7]:
# after removing duplicates
print(len(final_train_feats_list), len(final_macro_feats_list))

3 3


## Prepare data for Linear Regression

In [8]:
y_train = train_macro["price_doc"].to_numpy()

results = dict()
for train_feats,macro_feats in itertools.product(final_train_feats_list, final_macro_feats_list):
    # combine list of train_feats and macro_feats
    feats = train_feats + macro_feats
    
    # subset train_macro get only train_feats+macro+feats columns
    X_train = pd.get_dummies(train_macro).filter(feats, axis=1)
    
    reg = LinearRegression(n_jobs=-1)
    
    rmses = cross_val_score(reg, X_train, y_train, scoring='neg_root_mean_squared_error', n_jobs=-1)
    
    key = abs(rmses).mean()
    value = feats
    
    results[key] = value

In [9]:
min_rmse = min(list(results.keys()))
print("Min RMSE:", min_rmse)

feature_list = results[min_rmse] # feature list that gave us the smallest RMSE
print("Associated Feature List:")
print(feature_list)

Min RMSE: 0.9976251350098544
Associated Feature List:
['num_room', 'full_sq', 'sport_count_5000', 'sport_count_3000', 'trc_count_5000', 'zd_vokzaly_avto_km', 'sadovoe_km', 'kremlin_km', 'bulvar_ring_km', 'sport_count_2000', 'ttk_km', 'office_sqm_5000', 'trc_sqm_5000', 'sport_count_1500', 'nuclear_reactor_km', 'sport_objects_raion', 'trc_count_3000', 'cafe_count_5000_price_1000', 'stadium_km', 'cafe_count_5000_price_1500', 'cafe_count_5000', 'cafe_count_5000_na_price', 'ecology_no data', 'cafe_count_5000_price_500', 'office_sqm_3000', 'cafe_count_5000_price_2500', 'trc_sqm_3000', 'basketball_km', 'office_km', 'detention_facility_km', 'office_count_5000', 'university_km', 'office_sqm_2000', 'theater_km', 'cafe_count_5000_price_high', 'church_count_5000', 'swim_pool_km', 'catering_km', 'thermal_power_plant_km', 'cafe_count_5000_price_4000', 'bandwidth_sports', 'fixed_basket', 'provision_doctors', 'cpi', 'gdp_annual_growth', 'salary', 'gdp_deflator', 'load_of_teachers_school_per_teacher', 

Let's look at the R^2 score of the best model. 

In [34]:
X_train = pd.get_dummies(train_macro).filter(feature_list, axis=1)
y_train = train_macro["price_doc"]

reg = LinearRegression()

reg.fit(X_train, y_train)

LinearRegression()

In [36]:
reg.score(X_train, y_train)

0.37579998494716493

Save the feature set and the price_doc column to a file. 

In [17]:
train_macro_lemar_cols = pd.get_dummies(train_macro)[feature_list + ["price_doc"]] # also add price_doc to features for analysis in R

In [18]:
train_macro_lemar_cols.to_csv("../train_macro_lemar_cols.csv", index=False)

After a forward/backward stepwise regression in R (notebook 2-Feature-Selection-1.Rmd) these columns were selected:

In [26]:
selected_features = ["num_room","sport_count_5000","full_sq","salary","nuclear_reactor_km","office_sqm_5000",
                     "office_count_5000","cafe_count_5000_price_1500","office_sqm_3000","swim_pool_km",
                     "sport_count_2000","nuclear_reactor_km","cafe_count_5000_price_2500","cafe_count_5000",
                     "cafe_count_5000_price_4000","cafe_count_5000_price_high","zd_vokzaly_avto_km","kremlin_km",
                     "sadovoe_km","trc_count_3000","cpi","sport_objects_raion","ttk_km","thermal_power_plant_km",
                     "bulvar_ring_km","sport_count_3000","sport_count_5000","trc_sqm_5000",
                     "cafe_count_5000_na_price","sport_count_2000","ecology_no.data","church_count_5000",
                     "catering_km","turnover_catering_per_cap","salary","stadium_km","provision_doctors",
                     "trc_sqm_3000","trc_count_5000","trc_count_3000","office_km","basketball_km",
                     "cafe_count_5000_price_500"]

In [27]:
X_train = pd.get_dummies(train_macro).filter(selected_features, axis=1)
y_train = train_macro["price_doc"]

reg = LinearRegression()

reg.fit(X_train, y_train)

LinearRegression()

In [33]:
reg.score(X_train, y_train)

0.3743085015128025

In [30]:
abs(cross_val_score(reg, X_train, y_train, scoring='neg_root_mean_squared_error', n_jobs=-1)).mean()

0.9954358885658439

In [37]:
pd.get_dummies(train_macro)["cafe_count_5000_price_500"]

0       -0.215953
1       -0.136546
2       -0.295361
3       -0.470058
4        3.968828
           ...   
30466   -0.025375
30467    3.841776
30468   -0.493880
30469   -0.049197
30470   -0.319183
Name: cafe_count_5000_price_500, Length: 30471, dtype: float64

In [38]:
pd.get_dummies(train_macro).shape

(30471, 405)